Set up

In [1]:
from dataclasses import dataclass
from pathlib import Path
import math

import keras as tfk
from keras import layers as tfkl

import numpy as np
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.regularizers import l2
from tensorflow.data import Dataset as tfds

sns.set_theme()

In [2]:
SEED = 42
BATCH_SIZE = 512
METRICS = ["accuracy", "recall", "f1_score"]

tfk.utils.set_random_seed(SEED)

In [11]:
@dataclass
class Hyperparameters:
    # Model
    activation = "relu"
    # Training
    noise_std: float = 0.075
    optimiser = tfk.optimizers.Lion
    learning_rate1 = 1e-5
    learning_rate2 = 1e-6
    loss = tfk.losses.CategoricalCrossentropy()
    epochs = 200
    ## Early stopping parameters
    es_patience = 10
    es_min_delta = 1e-5
    ## Learning rate schedule
    lr_patience = 10
    lr_decay_factor = 0.1
    min_lr = 1e-7

hp = Hyperparameters()

Load augmented data

In [8]:
input_path = "/kaggle/input/aug-blood-cells"
train_dataset_path = input_path + "/augmented/train"
val_dataset_path = input_path + "/augmented/val"

augmented_train_dataset = tfds.load(train_dataset_path)
augmented_val_dataset = tfds.load(val_dataset_path)

augmented_train_dataset = augmented_train_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
augmented_val_dataset = augmented_val_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

Build the network

In [9]:
vgg19 = tfk.applications.VGG19(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=(96,96,3),
    pooling='avg'
)

80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [12]:
vgg19.trainable = False

inputs = tfkl.Input((96, 96, 3))
x = tfkl.GaussianNoise(hp.noise_std)(inputs)
x = vgg19(x)
x = tfkl.BatchNormalization()(x)
x = tfkl.Dense(512, activation=hp.activation)(x)
x = tfkl.Dense(256, activation=hp.activation)(x)
outputs = tfkl.Dense(8, activation="softmax")(x)

model = tfk.Model(inputs,outputs)


model.compile(loss=hp.loss,
              optimizer=hp.optimiser(learning_rate=hp.learning_rate1), metrics=['accuracy'])

Train the network

In [13]:
tl_history = model.fit(
    x=augmented_train_dataset,
    batch_size=BATCH_SIZE,
    epochs=hp.epochs,
    validation_data=augmented_val_dataset,
    callbacks=[tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=hp.es_patience, restore_best_weights=True)]
).history

Epoch 1/200


I0000 00:00:1732025223.250217     112 service.cc:145] XLA service 0x7d9278001870 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732025223.250282     112 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1732025246.894158     112 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


62/62 ━━━━━━━━━━━━━━━━━━━━ 82s 927ms/step - accuracy: 0.1704 - loss: 2.2302 - val_accuracy: 0.2425 - val_loss: 2.0117
Epoch 2/200
62/62 ━━━━━━━━━━━━━━━━━━━━ 16s 258ms/step - accuracy: 0.2887 - loss: 1.9396 - val_accuracy: 0.3707 - val_loss: 1.7942
Epoch 3/200
62/62 ━━━━━━━━━━━━━━━━━━━━ 16s 258ms/step - accuracy: 0.3979 - loss: 1.7500 - val_accuracy: 0.4621 - val_loss: 1.6290
Epoch 4/200
62/62 ━━━━━━━━━━━━━━━━━━━━ 16s 259ms/step - accuracy: 0.4809 - loss: 1.5824 - val_accuracy: 0.5288 - val_loss: 1.4821
Epoch 5/200
62/62 ━━━━━━━━━━━━━━━━━━━━ 16s 259ms/step - accuracy: 0.5440 - loss: 1.4314 - val_accuracy: 0.5705 - val_loss: 1.3606
Epoch 6/200
62/62 ━━━━━━━━━━━━━━━━━━━━ 16s 259ms/step - accuracy: 0.5861 - loss: 1.3028 - val_accuracy: 0.5987 - val_loss: 1.2522
Epoch 7/200
62/62 ━━━━━━━━━━━━━━━━━━━━ 16s 259ms/step - accuracy: 0.6166 - loss: 1.1886 - val_accuracy: 0.6268 - val_loss: 1.1555
Epoch 8/200
62/62 ━━━━━━━━━━━━━━━━━━━━ 16s 259ms/step - accuracy: 0.6475 - loss: 1.0883 - val_accuracy

In [14]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 96, 96, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gaussian_noise (GaussianNoise)  │ (None, 96, 96, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg19 (Functional)              │ (None, 512)            │    20,024,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 8)              │         2,056 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,819,538 (79.42 MB)

 Trainable params: 397,064 (1.51 MB)

 Non-trainable params: 20,025,408 (76.39 MB)

 Optimizer params: 397,066 (1.51 MB)

In [15]:
vgg19.trainable = True

for layer in vgg19.layers[:15]:
    layer.trainable = False

model.compile(loss=hp.loss,
              optimizer=hp.optimiser(learning_rate=hp.learning_rate1), metrics=['accuracy'])

history_fine_tuning = model.fit(
    x=augmented_train_dataset,
    batch_size=BATCH_SIZE,
    epochs=hp.epochs,
    validation_data=augmented_val_dataset,
    callbacks=[tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=hp.es_patience, restore_best_weights=True)]
).history

Epoch 1/200
62/62 ━━━━━━━━━━━━━━━━━━━━ 37s 447ms/step - accuracy: 0.9923 - loss: 0.0378 - val_accuracy: 0.9176 - val_loss: 0.4062
Epoch 2/200
62/62 ━━━━━━━━━━━━━━━━━━━━ 20s 324ms/step - accuracy: 0.9921 - loss: 0.0277 - val_accuracy: 0.9274 - val_loss: 0.3777
Epoch 3/200
62/62 ━━━━━━━━━━━━━━━━━━━━ 20s 326ms/step - accuracy: 0.9905 - loss: 0.0306 - val_accuracy: 0.9419 - val_loss: 0.2618
Epoch 4/200
62/62 ━━━━━━━━━━━━━━━━━━━━ 20s 327ms/step - accuracy: 0.9933 - loss: 0.0211 - val_accuracy: 0.9422 - val_loss: 0.2535
Epoch 5/200
62/62 ━━━━━━━━━━━━━━━━━━━━ 20s 327ms/step - accuracy: 0.9953 - loss: 0.0133 - val_accuracy: 0.9557 - val_loss: 0.1965
Epoch 6/200
62/62 ━━━━━━━━━━━━━━━━━━━━ 20s 328ms/step - accuracy: 0.9964 - loss: 0.0114 - val_accuracy: 0.9612 - val_loss: 0.1707
Epoch 7/200
62/62 ━━━━━━━━━━━━━━━━━━━━ 20s 327ms/step - accuracy: 0.9970 - loss: 0.0086 - val_accuracy: 0.9710 - val_loss: 0.1409
Epoch 8/200
62/62 ━━━━━━━━━━━━━━━━━━━━ 20s 326ms/step - accuracy: 0.9974 - loss: 0.0078 - 

In [16]:
vgg19.trainable = True

for layer in vgg19.layers[:12]:
    layer.trainable = False

model.compile(loss=hp.loss,
              optimizer=hp.optimiser(learning_rate=hp.learning_rate2), metrics=['accuracy'])

history_fine_tuning = model.fit(
    x=augmented_train_dataset,
    batch_size=BATCH_SIZE,
    epochs=hp.epochs,
    validation_data=augmented_val_dataset,
    callbacks=[tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=hp.es_patience, restore_best_weights=True)]
).history

Epoch 1/200
62/62 ━━━━━━━━━━━━━━━━━━━━ 28s 378ms/step - accuracy: 0.9993 - loss: 0.0017 - val_accuracy: 0.9832 - val_loss: 0.1356
Epoch 2/200
62/62 ━━━━━━━━━━━━━━━━━━━━ 20s 323ms/step - accuracy: 1.0000 - loss: 1.4437e-04 - val_accuracy: 0.9846 - val_loss: 0.1208
Epoch 3/200
62/62 ━━━━━━━━━━━━━━━━━━━━ 20s 324ms/step - accuracy: 1.0000 - loss: 5.8943e-05 - val_accuracy: 0.9855 - val_loss: 0.1174
Epoch 4/200
62/62 ━━━━━━━━━━━━━━━━━━━━ 20s 325ms/step - accuracy: 1.0000 - loss: 4.6680e-05 - val_accuracy: 0.9850 - val_loss: 0.1245
Epoch 5/200
62/62 ━━━━━━━━━━━━━━━━━━━━ 20s 326ms/step - accuracy: 1.0000 - loss: 3.1559e-05 - val_accuracy: 0.9855 - val_loss: 0.1209
Epoch 6/200
62/62 ━━━━━━━━━━━━━━━━━━━━ 20s 326ms/step - accuracy: 1.0000 - loss: 1.6997e-05 - val_accuracy: 0.9866 - val_loss: 0.1214
Epoch 7/200
62/62 ━━━━━━━━━━━━━━━━━━━━ 20s 327ms/step - accuracy: 1.0000 - loss: 8.4969e-06 - val_accuracy: 0.9869 - val_loss: 0.1281
Epoch 8/200
62/62 ━━━━━━━━━━━━━━━━━━━━ 20s 326ms/step - accuracy: 

In [19]:
output_path = "/kaggle/working/vgg19-lion.keras"

model.save(output_path)

In [18]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 96, 96, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gaussian_noise (GaussianNoise)  │ (None, 96, 96, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg19 (Functional)              │ (None, 512)            │    20,024,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 8)              │         2,056 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,618,578 (124.43 MB)

 Trainable params: 12,196,104 (46.52 MB)

 Non-trainable params: 8,226,368 (31.38 MB)

 Optimizer params: 12,196,106 (46.52 MB)